## Parsing example

Install the lark-parser with

> pip install lark-parser 

### Expression Grammar definition

In [13]:
from lark import Lark, InlineTransformer 

grammar_exp = """
    ?start: exp
    ?exp: product
        | exp "+" product   -> add
        | exp "-" product   -> sub
    ?product: atom
        | product "*" atom  -> mul
        | product "/" atom  -> div
    ?atom: NUMBER           -> number
         | "-" atom         -> neg
         | NAME             -> var
         | "(" exp ")"
    %import common.CNAME -> NAME
    %import common.NUMBER
    %import common.WS_INLINE
    %ignore WS_INLINE
"""

Implementing the function described by grammar productions

In [14]:
class Tree_Exp(InlineTransformer):
    number = float

    def number(self, value):
        return ['number_const', float(value)]

    def var(self, name):
        return ['variable', str(name)]
    
    def add(self, left, right):
        return ['+', left, right]
    
    def sub(self, left, right):
        return ['-', left, right]
    
    def mul(self, left, right):
        return ['*', left, right]
    
    def div(self, left, right):
        return ['/', left, right]
    
    def neg(self, left):
        return ['-', left]

Building the parser using the defined AST representation

In [15]:
def get_parser_exp():
    parser = Lark(grammar_exp, parser='lalr', transformer=Tree_Exp())
    return parser.parse

Example of usage

In [16]:
parse = get_parser_exp()
    
E1 = parse("3+5")
E1

['+', ['number_const', 3.0], ['number_const', 5.0]]

In [5]:
E2 = parse("2*x-y")
E2

['-', ['*', ['number_const', 2.0], ['variable', 'x']], ['variable', 'y']]

In [6]:
E3 = parse("x*(5+z)")
E3

['*', ['variable', 'x'], ['+', ['number_const', 5.0], ['variable', 'z']]]

### Objective: define solve_exp(E, sigma)

How to get to a function that given an AST and a sigma evaluate the Expression in sigma?

In [9]:
sigma = {} # empty interpretation
# print('E1 = ', solve_exp(E1, sigma))

In [10]:
sigma = { 'x':2, 'y':1 }
# print('E2 = ', solve_exp(E2, sigma))

In [11]:
sigma = { 'x':2, 'y':1, 'z':1 }
# print('E3 = ', solve_exp(E3, sigma))

Let's define a solver

In [27]:
def solve_exp(exp, sigma):
    op = exp[0]
    if op == 'number_const':
        return exp[1]
    if op == 'variable':
        return sigma[exp[1]]
    if op == '+':
        return solve_exp(exp[1], sigma) + solve_exp(exp[2], sigma)
    if op == '-':
        return solve_exp(exp[1], sigma) - solve_exp(exp[2], sigma)
    if op == '/':
        return solve_exp(exp[1], sigma) / solve_exp(exp[2], sigma)
    if op == '*':
        return solve_exp(exp[1], sigma) * solve_exp(exp[2], sigma)
    if op == '-':
        return -solve_exp(exp[1], sigma)

Using solve_exp to move from expression to their evaluation

Evaluating a number

In [22]:
E0 = parse("5")
print(E0)
solve_exp(E0, sigma)

['number_const', 5.0]


5.0

Evaluating a variable (given the current sigma)

In [24]:
E0 = parse("x")
print(E0)
solve_exp(E0, sigma)

['variable', 'x']


2

Evaluating a sum

In [26]:
E0 = parse("x+y")
print(E0)
solve_exp(E0, sigma)

['+', ['variable', 'x'], ['variable', 'y']]


3

Evaluating a composite expression

In [28]:
E0 = parse("x+y-5*2/6")
print(E0)
solve_exp(E0, sigma)

['-', ['+', ['variable', 'x'], ['variable', 'y']], ['/', ['*', ['number_const', 5.0], ['number_const', 2.0]], ['number_const', 6.0]]]


1.3333333333333333